In [2]:
import pandas as pd
import polars as pl
import gc
import json

In [3]:
def j2d(f):
    with open(f, "rt") as file:
        d = json.load(file)
    return d

def d2j(d, f):
    with open(f, "wt") as file:
        json.dump(d, file, indent = 2)

In [4]:
l_cancer = j2d("../../share3/mat/cans_codes_icd.json")['icdcode']
tag_cancer = set(l_cancer)
l_cancer[:3]

['C00', 'C01', 'C02']

In [5]:
l_ = j2d("../../share3/mat/cans_codes.json")['cancercode']
code_cancer = set(l_)

l_[:3]

['8845943', '8845270', '8844174']

In [6]:
# l_cancer = ["C" + str(i).zfill(2) for i in range(0, 100)] + \
#         ["D" + str(i).zfill(2) for i in range(0, 9)] + \
#         ["D" + str(i).zfill(2) for i in range(37, 49)]
# tag_cancer = set(l_cancer)
# # l_cancer = sorted(list(set(l_cancer)))
# # tag_cancer
# # l_cancer

In [7]:
# s_crc = set(["C18" ,"C19","C20", "C21"])
s_crc = set(["C18" ,"C19","C20"])
s_bld = set(["C23","C24"])
s_lung = set(["C33","C34"])
# s_kdn = set(["C64", "C65", "C66", "C68"])
s_skin = set(["C43","C44"])
s_2nd =  set(["C78","C79"])
s_lym =  set(["C81","C82","C83","C84","C85","C86"])
s_leu =  set(["C91","C92","C93","C94","C95"])

In [8]:
f = "../../share3/m_icd10.parquet"

m_dis = pl.scan_parquet(f)\
    .filter(
        pl.col("icd10_sub_code").is_in(tag_cancer)
    )\
    .select(
        pl.col("diseases_code", "icd10_kbn_code", "icd10_code","icd10_sub_code")
    )\
    .with_columns(
        pl.when(pl.col("icd10_sub_code").is_in(s_crc))
        .then(pl.lit("C18-C20"))
        .when(pl.col("icd10_sub_code").is_in(s_bld))
        .then(pl.lit("C23-C24"))
        .when(pl.col("icd10_sub_code").is_in(s_lung))
        .then(pl.lit("C33-C34"))
        # .when(pl.col("icd10_sub_code").is_in(s_kdn))
        # .then(pl.lit("C64-C66 and C68"))
        .when(pl.col("icd10_sub_code").is_in(s_skin))
        .then(pl.lit("C43-C44"))
        .when(pl.col("icd10_sub_code").is_in(s_2nd))
        .then(pl.lit("C78-C79"))
        .when(pl.col("icd10_sub_code").is_in(s_lym))
        .then(pl.lit("C81-C86"))
        .when(pl.col("icd10_sub_code").is_in(s_leu))
        .then(pl.lit("C91-C95"))
        .otherwise(pl.col("icd10_sub_code"))
        .alias("icd10_sub_code_fix")
    )\
    .select(pl.exclude("icd10_sub_code"))\
    .rename({"icd10_sub_code_fix":"icd10_sub_code"})
# code_cancer = m_dis\
#     .collect()\
#     .select(
#             pl.col("diseases_code")
#     ).to_series()\
#     .unique().to_list()

# f_dcans = "../../share3/mat/cans_codes.json"
# d2j(d={"cancercode":l_cancer}, f=f_dcans)

# code_cancer = set(code_cancer )
# m_dis.fetch()

In [9]:
# m_dis\
#     .filter(pl.col("icd10_sub_code") != pl.col("icd10_sub_code_fix"))\
#     .collect()

In [10]:
lx = [str(i) for i in range(10)]
lx

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [11]:
%%time
        # .agg(
        #     (pl.col("icd10_sub_code") == value).max().alias(value)
        #     for value in l_cancer
        # )\


for i in lx:
    f = "../../share3/dis/df_dis_split_{}.parquet".format(i)
    dis_ = pl.scan_parquet(f)\
        .filter(pl.col("utagai_flg") != "1")\
        .filter(pl.col("diseases_code").is_in(code_cancer))\
        .join(other=m_dis, 
              how="left",
              on= "diseases_code")\
        .group_by("kojin_id","grp", "icd10_sub_code")\
        .agg(
            pl.col("receipt_ym").str.replace("/", "").cast(pl.Int32).min().cast(pl.String).alias("min_ym"))\
        .with_columns(
            pl.lit(True).alias("with_cancer_diag_codes"),
            (pl.col("min_ym").str.slice(0,4) +
             "/" + 
             pl.col("min_ym").str.slice(4,6)).alias("receipt_ym")
        )\
        .select(pl.exclude("min_ym"))
        


    
        # .group_by("kojin_id","grp", "receipt_ym")\
        # .agg(
        #     pl.col("icd10_sub_code").unique().sort().str.concat(",").alias("cancers"))\
        # .with_columns(
        #     pl.lit(True).alias("with_cancer_diag_codes"))
        
    

    f_out = "../../share3/diag/df_cancers_diag_split_{}.parquet".format(i)
    dis_.collect(streaming=True).write_parquet(f_out)
    # dis_.sink_parquet(f_out)
    print(f_out)
    gc.collect()
    # break

../../share3/diag/df_cancers_diag_split_0.parquet
../../share3/diag/df_cancers_diag_split_1.parquet
../../share3/diag/df_cancers_diag_split_2.parquet
../../share3/diag/df_cancers_diag_split_3.parquet
../../share3/diag/df_cancers_diag_split_4.parquet
../../share3/diag/df_cancers_diag_split_5.parquet
../../share3/diag/df_cancers_diag_split_6.parquet
../../share3/diag/df_cancers_diag_split_7.parquet
../../share3/diag/df_cancers_diag_split_8.parquet
../../share3/diag/df_cancers_diag_split_9.parquet
CPU times: user 16min 55s, sys: 44 s, total: 17min 39s
Wall time: 3min 36s


In [21]:
# dis_.collect()
